In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os import getcwd
import shutil
import zipfile
from shutil import copyfile
import random

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
print(os.listdir("../input"))

In [ ]:
cat_dog_path = "../input/dogs-vs-cats"
os.listdir(cat_dog_path)

*Extracting the Train Images*

In [ ]:
train_zip = "../input/dogs-vs-cats/train.zip"
zip_file = zipfile.ZipFile(train_zip,'r')
zip_file.extractall('.')
zip_file.close()

*Extracting the test images*

In [ ]:
test_zip = "../input/dogs-vs-cats/test1.zip"
zip_file = zipfile.ZipFile(train_zip,'r')
zip_file.extractall('.')
zip_file.close()

In [ ]:
try:
    main_dir = "/kaggle/working/"
    train_dir = "train"
    test_dir = "test"
    train_dir = os.path.join(main_dir,train_dir)
    
    # Directory with our training cat/dog pictures
    train_cats_dir = os.path.join(train_dir, 'cats')
    train_dogs_dir = os.path.join(train_dir, 'dogs')
    os.mkdir(train_cats_dir)
    os.mkdir(train_dogs_dir)
    
    # Directory with our testing cat/dog pictures
    test_dir = os.path.join(main_dir,"test")
    os.mkdir(test_dir)
    test_cats_dir = os.path.join(test_dir, 'cats')
    test_dogs_dir = os.path.join(test_dir, 'dogs')
    os.mkdir(test_cats_dir)
    os.mkdir(test_dogs_dir)

except OSError:
    pass

Creating a "working" directory for storing the extracted images of cats and dogs.
<br>
<br>

In the "working" directory a folder "train" is created. The images in the dataset are have there labels in their names itself. So the image will stored in the respective cats and dogs folder created in the "train" folder depending in their starting 3 characters of their names.

In [ ]:
main_dir = "/kaggle/working/"
train_dir = "train"
train_path = os.path.join(main_dir,train_dir)

prefixed_dogs = [filename for filename in os.listdir(train_path) if filename.startswith("dog.")]
print(len(prefixed_dogs))
prefixed_cats = [filename for filename in os.listdir(train_path) if filename.startswith("cat.")]
print(len(prefixed_cats))

def move_files(src_file):
    
    for filename in prefixed_dogs:
        shutil.move(src_file+filename, src_file+'dogs/'+filename)
        
    for filename in prefixed_cats:
        shutil.move(src_file+filename, src_file+'cats/'+filename)
    

move_files("/kaggle/working/train/")

In [ ]:
print(len(os.listdir('/kaggle/working/train/dogs')))
print(len(os.listdir('/kaggle/working/train/cats')))
print(len(os.listdir('/kaggle/working/train')))

The "train" folder has teo sub folders "cats" and "dogs" with 12500 images each.
<br>
We are splitting the images in both the folder to create a test set for each class.

In [ ]:
def split_data(SOURCE, TEST, SPLIT_SIZE):

    SRC_files = [f for f in os.listdir(SOURCE) if os.path.isfile(os.path.join(SOURCE, f))]
    SRC_Size = len(SRC_files)
    #print(SRC_Size)
    if SRC_Size != 0:
        # we shuffle the images before the split
        shuffled_files = random.sample(SRC_files, len(SRC_files))
        #print("shuffled")
        TRN_size = int(SRC_Size * SPLIT_SIZE)
        TEST_SIZE = int(SRC_Size - TRN_size)
        print(TRN_size)
        train_set = shuffled_files[0:TRN_size]
        test_set = shuffled_files[-TEST_SIZE:SRC_Size]
        for filename in test_set:
            if os.path.getsize(SOURCE+filename)!=0:
                shutil.move(SOURCE+filename, TEST+filename)
            else:
                print(filename + ' is zero length. So ignoring!')
                pass


                    
CAT_SOURCE_DIR = "/kaggle/working/train/cats/"
TESTING_CATS_DIR = "/kaggle/working/test/cats/"

DOG_SOURCE_DIR = "/kaggle/working/train/dogs/"
TESTING_DOGS_DIR = "/kaggle/working/test/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TESTING_DOGS_DIR, split_size)

In [ ]:
print(len(os.listdir('/kaggle/working/train/dogs')))
print(len(os.listdir('/kaggle/working/train/cats')))
print(len(os.listdir('/kaggle/working/train')))
print(len(os.listdir('/kaggle/working/test/dogs')))
print(len(os.listdir('/kaggle/working/test/cats')))

So finally we have <br>

***Working Directory***
    <ul>
        <li><b>Train</b></li>
        <ul><li>Cats - 11250 Images</li>
            <li>Dogs - 11250 Images</li>
        </ul>
        </ul>
    <ul>
    <li><b>Test</b></li>
    <ul><li>Cats - 1250 Images</li>
        <li>Dogs - 1250 Images</li>
    </ul>
    </ul>


In [ ]:
from matplotlib.image import imread

In [ ]:
sample_dog_image = '/kaggle/working/train/dogs/dog.7024.jpg'
plt.imshow(imread(sample_dog_image))

In [ ]:
train_path = "/kaggle/working/train/"
test_path="/kaggle/working/test/"

In [ ]:
image_width = 128
image_height = 128
image_color = 3
image_shape = (image_width,image_height,image_color)
batch_size = 32

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from numpy import expand_dims

In [ ]:
sample_img_gen = ImageDataGenerator(rotation_range=20,
                                    height_shift_range=0.06,
                                    width_shift_range=0.06,
                                    shear_range=0.1,
                                    horizontal_flip=True,
                                    zoom_range=[0.5,0.8])

sample_image = load_img(sample_dog_image)
sample_image_array = img_to_array(sample_image)
test_image = expand_dims(sample_image_array,0)
image_aug = sample_img_gen.flow(test_image,batch_size=1)

In [ ]:
for i in range(9):
    plt.subplot(330+1+i)
    batch = image_aug.next()
    image = batch[0].astype('uint8')
    plt.imshow(image)
plt.show()

In [ ]:
for i in range(9):
    plt.subplot(330+1+i)
    batch = image_aug.next()
    image = batch[0].astype('uint8')
    plt.imshow(image)
plt.show()

In [ ]:
train_image_gen = ImageDataGenerator(rotation_range=20,
                                     rescale=1./255,
                                     height_shift_range=0.06,
                                     width_shift_range=0.06,
                                     shear_range=0.1,
                                     horizontal_flip=True,
                                     zoom_range=[0.6,0.8])

train_data = train_image_gen.flow_from_directory(train_path,
                                                 target_size=image_shape[:2],
                                                 color_mode = 'rgb',
                                                 batch_size = batch_size,
                                                 class_mode='binary')
print(train_data.class_indices)

In [ ]:
test_img_gen = ImageDataGenerator(rescale=1./255)

test_data = test_img_gen.flow_from_directory(test_path,
                                             target_size=image_shape[:2],
                                             color_mode = 'rgb',
                                             batch_size = batch_size,
                                             class_mode='binary')
print(test_data.class_indices)

In [ ]:
length_train = 22500
length_test = 2500

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(2,2), activation='relu',padding='same', input_shape=image_shape))
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(units=2,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=4)

In [ ]:
model.fit_generator(train_data,validation_data=test_data,
                   steps_per_epoch = length_train//batch_size,
                   validation_steps= length_test//batch_size,
                   epochs=15,callbacks=[early_stop])

In [ ]:
result = pd.DataFrame(model.history.history)

In [ ]:
result[['loss','val_loss']].plot()

In [ ]:
result[['accuracy','val_accuracy']].plot()
